In [5]:
import sys, os
working_directory = os.path.dirname(sys.path[0])
sys.path.append(os.path.dirname(sys.path[0]))
print(working_directory)

/home/user/jacobs/cudavision


In [64]:
import shutil
from download import download

In [65]:
download(working_directory)

Beginning file download with urllib2...
Creating /home/user/jacobs/cudavision/blob/dataset/
Creating /home/user/jacobs/cudavision/blob/forceTest/
Creating /home/user/jacobs/cudavision/blob/forceTrain/
Creating /home/user/jacobs/cudavision/segmentation/dataset/image/
Creating /home/user/jacobs/cudavision/segmentation/dataset/target/
Creating /home/user/jacobs/cudavision/segmentation/forceTrain/image/
Creating /home/user/jacobs/cudavision/segmentation/forceTrain/target/


In [34]:
data_dir = os.path.join(working_directory,'data')
if os.path.isdir(data_dir):
    print("Warning. The following procedure assumes " + data_dir + " is empty.")
else:
    os.makedirs(data_dir)

Warning. The following procedure assumes /home/user/jacobs/cudavision/data is empty.


In [35]:
subdirs = []
for split in ['Train','Test','Validation']:
    subdirs.append(os.path.join(data_dir,split))
categories = ['segmentation','image','detection']

for subdir in subdirs:
    print(subdir)
    os.makedirs(subdir)
    for categorie in categories:
        print(os.path.join(subdir,categorie))
        os.makedirs(os.path.join(subdir,categorie))

/home/user/jacobs/cudavision/data/Train
/home/user/jacobs/cudavision/data/Train/segmentation
/home/user/jacobs/cudavision/data/Train/image
/home/user/jacobs/cudavision/data/Train/detection
/home/user/jacobs/cudavision/data/Test
/home/user/jacobs/cudavision/data/Test/segmentation
/home/user/jacobs/cudavision/data/Test/image
/home/user/jacobs/cudavision/data/Test/detection
/home/user/jacobs/cudavision/data/Validation
/home/user/jacobs/cudavision/data/Validation/segmentation
/home/user/jacobs/cudavision/data/Validation/image
/home/user/jacobs/cudavision/data/Validation/detection


In [36]:
for origin in ['blob/dataset','blob/forceTrain','blob/forceTest']:
    sourcepath= os.path.join(working_directory,origin)
    sourcefiles = os.listdir(sourcepath)
    destinationpath_image = os.path.join(data_dir,'Train/image')
    destinationpath_detection = os.path.join(data_dir,'Train/detection')
    for file in sourcefiles:
        if file.endswith('.png') or file.endswith('.jpg'):
            shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath_image,file))
        else:
            if file.endswith('.xml'):
                shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath_detection,file))
                
def move_files(source_path,destination_path):
    source_files = os.listdir(source_path)
    for file in source_files:
        shutil.move(os.path.join(source_path,file), os.path.join(destination_path,file))
        
move_files(os.path.join(working_directory,'segmentation/forceTrain/image'),os.path.join(data_dir,'Train/image'))
move_files(os.path.join(working_directory,'segmentation/forceTrain/target'),os.path.join(data_dir,'Train/segmentation'))
move_files(os.path.join(working_directory,'segmentation/dataset/image'), os.path.join(data_dir,'Train/image'))
move_files(os.path.join(working_directory,'segmentation/dataset/target'), os.path.join(data_dir,'Train/segmentation'))

In [37]:
folder_names = ['blob/dataset','blob/forceTest','blob/forceTrain','segmentation/dataset/image','segmentation/dataset/target','segmentation/forceTrain/image','segmentation/forceTrain/target']
for folder_name in folder_names:
    os.rmdir(os.path.join(working_directory,folder_name))
folder_names = ['blob','segmentation/dataset','segmentation/forceTrain']
for folder_name in folder_names:
    os.rmdir(os.path.join(working_directory,folder_name))
os.rmdir(os.path.join(working_directory,'segmentation'))

In [78]:
# Need to delete images, which are duplicate
destinationpath_image = os.path.join(data_dir,'Train/image')
destination_files = os.listdir(destinationpath_image)
destination_files_no_end = []
for file in destination_files:
    destination_files_no_end.append(file[:-4])
count = Counter(destination_files_no_end)
for name in count:
    if count[name] > 1:
        os.remove(os.path.join(destinationpath_image,name+'.jpg'))

In [79]:
# Create Split

In [80]:
from RobotData import RobotsDataset
import pandas as pd

In [82]:
dataset = RobotsDataset(data_loc = os.path.join(data_dir,'Train'),mode = 'det',transform=None)

Generated both csv with 1110 entries
Generated seg csv with 1192 entries
Generated det csv with 8851 entries


In [83]:
index = {'det': dataset.index_det,
         'seg': dataset.index_seg,
         'both': dataset.index_both
        }

In [230]:
import random
index_all = pd.merge(index['det'], index['seg'], on=['name'], how='outer', indicator=True).filter(['name','_merge'])
for mode in ['both', 'left_only', 'right_only']:
    subset = index_all['_merge'] == mode
    num_samples = len(index_all[subset])
    split_values = ['train']*int(0.7*num_samples) + ['test']*int(0.2*num_samples) + ['validation']* (num_samples - int(0.7*num_samples) - int(0.3*num_samples))
    random.shuffle(split_values)
    index_all.loc[subset,'split'] = split_values
    
index_all.set_index('name', inplace = True)
index_all.rename(columns={'_merge': 'mode'}, inplace=True)
index_all['mode'].replace('left_only','det', inplace = True)
index_all['mode'].replace('right_only','seg', inplace = True)

In [205]:
# Move data into directories
def get_name(file):
    return file[:-4]
source_path = os.path.join(data_dir,'Train/image')
source = os.listdir(source_path)
missing = []
for file in source:
    if get_name(file) in index_all.index:
        if index_all.loc[get_name(file),'split'] == 'test':
            destination_path = os.path.join(data_dir,'Test/image')
            shutil.move(os.path.join(source_path,file), os.path.join(destination_path,file))
        if index_all.loc[get_name(file),'split'] == 'validation':
            destination_path = os.path.join(data_dir,'Validation/image')
            shutil.move(os.path.join(source_path,file), os.path.join(destination_path,file))
    else:
        missing.append(file)   
        
source_path = os.path.join(data_dir,'Train/segmentation')
source = os.listdir(source_path)
missing_seg = []
for file in source:
    if get_name(file) in index_all.index:
        if index_all.loc[get_name(file),'split'] == 'test':
            destination_path = os.path.join(data_dir,'Test/segmentation')
            shutil.move(os.path.join(source_path,file), os.path.join(destination_path,file))
        if index_all.loc[get_name(file),'split'] == 'validation':
            destination_path = os.path.join(data_dir,'Validation/segmentation')
            shutil.move(os.path.join(source_path,file), os.path.join(destination_path,file))
    else:
        missing_seg.append(file)

source_path = os.path.join(data_dir,'Train/detection')
source = os.listdir(source_path)
missing_det = []
for file in source:
    if get_name(file) in index_all.index:
        if index_all.loc[get_name(file),'split'] == 'test':
            destination_path = os.path.join(data_dir,'Test/detection')
            shutil.move(os.path.join(source_path,file), os.path.join(destination_path,file))
        if index_all.loc[get_name(file),'split'] == 'validation':
            destination_path = os.path.join(data_dir,'Validation/detection')
            shutil.move(os.path.join(source_path,file), os.path.join(destination_path,file))
    else:
        missing_det.append(file)    

In [206]:
# Here the target is present, but no matching image
missing_det

['label.xml',
 '2018-06-17-15-16-11_.xml',
 '2018-06-17-15-17-38_.xml',
 '2018-06-17-15-13-20_.xml',
 '2018-06-17-23-06-10_.xml',
 '2018-06-17-15-17-52_.xml',
 'bs15_leipzig_robocup16_012.000.xml',
 'output_iHOP__0072_copy.xml',
 'frame0594_dropin.xml',
 'frame0504_dropin.xml',
 '2018-06-17-23-06-50_.xml',
 '2018-06-17-15-13-07_.xml',
 '2018-06-17-15-15-28_.xml',
 'bs15_leipzig_robocup16_011.650.xml',
 '2018-06-17-23-10-49_.xml',
 'frame0506_dropin.xml',
 '2018-06-17-15-17-44_.xml',
 'oleg_frame0362d.xml']

In [239]:
split = index_all.filter(['split'])
split.to_pickle(os.path.join(working_directory,'split.pkl'))

In [248]:
# Delete old index files
source_files = os.listdir(data_dir)
for file in source_files:
    if file.endswith('.csv'):
        os.remove(os.path.join(data_dir,file))

In [7]:
# Initialize meta data with the new split
dataset_dir = os.path.join(working_directory,'Robot_datasets')
meta = MetaData(dataset_dir = dataset_dir, split_df = split)